In [2]:
import pandas as pd
import numpy as np
from googleapiclient.discovery import build
from api_key import api_key
import re

In [3]:
#데이터프레임 불러오기
df = pd.read_excel('./category_5.xlsx', engine="openpyxl")

In [4]:
#댓글 추출 함수
def dfcomment(video_id_list,name_list,file_path):
    errorvid=[]
    concat_list=[]
    for video_id,name in zip(video_id_list,name_list):
        try:
            comments = list()
            api_obj = build('youtube', 'v3', developerKey=api_key)
            response = api_obj.commentThreads().list(part='snippet,replies', videoId=video_id, maxResults=100).execute()

            while response:
                for item in response['items']:
                    comment = item['snippet']['topLevelComment']['snippet']
                    comments.append([comment['textDisplay'], comment['authorDisplayName'], comment['publishedAt'], comment['likeCount']])

                    if item['snippet']['totalReplyCount'] > 0:
                        if 'replies' in item.keys():
                            for reply_item in item['replies']['comments']:
                                reply = reply_item['snippet']
                                comments.append([reply['textDisplay'], reply['authorDisplayName'], reply['publishedAt'], reply['likeCount']])

                if 'nextPageToken' in response:
                    response = api_obj.commentThreads().list(part='snippet,replies', videoId=video_id, pageToken=response['nextPageToken'], maxResults=100).execute()
                else:
                    break


            df = pd.DataFrame(comments)
            df['Video_Name']=name
            df['Channel']=f'{file_path[2:6]}'
            concat_list.append(df)
        except:
            print(video_id)
            continue

    return pd.concat(concat_list)

In [5]:
#댓글 추출시 필요한 변수들
video_id_list=list(df['동영상_ID'])
name_list=list(df['제목'])
file_path='./'
dfresult=dfcomment(video_id_list,name_list,file_path)

eNtKIvjo-70
tkvhZNYeMGA
o7BIXu6punw
IgIrLOyyp5o
MivKdTkUung


In [6]:
dfresult.to_excel('./댓글5.xlsx')

In [ ]:
dfresult